In [13]:
import requests
import pandas as pd
import time

anime_data = []

# Ambil 4 halaman → 100 anime
for page in range(1, 5):
    url = f"https://api.jikan.moe/v4/top/anime?page={page}"
    res = requests.get(url)
    data = res.json()

    for item in data['data']:
        anime_data.append({
            'title': item['title'],
            'type': item['type'],
            'score': item['score'],
            'episodes': item['episodes'],
            'members': item['members'],
            'rank': item['rank'],
            'popularity': item['popularity'],
            'start_date': item['aired']['from'][:10] if item['aired']['from'] else None
        })
    time.sleep(1)  # delay biar aman

# Masukkan ke DataFrame
df = pd.DataFrame(anime_data)
df = df.sort_values(by='rank').reset_index(drop=True)

# Tampilkan 5 data teratas
print(df.head())


                                title   type  score  episodes  members  rank  \
0                   Sousou no Frieren     TV   9.30      28.0  1172597   1.0   
1    Fullmetal Alchemist: Brotherhood     TV   9.10      64.0  3548367   2.0   
2                         Steins;Gate     TV   9.07      24.0  2717073   3.0   
3  Shingeki no Kyojin Season 3 Part 2     TV   9.05      10.0  2470344   4.0   
4                  Gintama: The Final  Movie   9.05       1.0   171969   5.0   

   popularity  start_date  
0         137  2023-09-29  
1           3  2009-04-05  
2          14  2011-04-06  
3          21  2019-04-29  
4        1524  2021-01-08  


In [14]:
df.to_csv('top_100_anime.csv', index=False)
print("Data berhasil disimpan ke top_100_anime.csv")


Data berhasil disimpan ke top_100_anime.csv


In [15]:
# Cek missing value
print("Missing value:\n", df.isnull().sum())

# Hapus baris dengan nilai kosong di kolom penting
df = df.dropna(subset=['score', 'episodes', 'start_date'])

# Konversi start_date ke datetime + ambil tahunnya
df['start_date'] = pd.to_datetime(df['start_date'])
df['year'] = df['start_date'].dt.year

# Reset index
df = df.reset_index(drop=True)

# Simpan versi bersih
df.to_csv('top_100_anime_clean.csv', index=False)
print("Data sudah dibersihkan dan disimpan ke top_100_anime_clean.csv")


Missing value:
 title         0
type          0
score         0
episodes      1
members       0
rank          2
popularity    0
start_date    0
dtype: int64
Data sudah dibersihkan dan disimpan ke top_100_anime_clean.csv


In [1]:
import dash
from dash import html, dcc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output

# Load cleaned anime data
df = pd.read_csv("top_100_anime_clean.csv")
df = df.sort_values(by='rank').reset_index(drop=True)

# Inisialisasi aplikasi Dash
app = dash.Dash(__name__)
app.title = "Dashboard Anime Top 100"

# Layout Aplikasi
app.layout = html.Div([
    html.H1("Dashboard Interaktif: Top 100 Anime Terbaik - MyAnimeList",
            style={'textAlign': 'center', 'color': '#5C5470'}),

    html.Div([
        html.H4(f"Jumlah Anime: {len(df)}"),
        html.H4(f"Skor Tertinggi: {df['score'].max()}"),
        html.H4(f"Skor Terendah: {df['score'].min()}"),
        html.H4(f"Tahun Rilis Paling Awal: {df['year'].min()}"),
        html.H4(f"Tahun Rilis Terbaru: {df['year'].max()}"),
    ], style={'textAlign': 'left', 'marginBottom': '30px'}),

    dcc.Dropdown(
        id='type-dropdown',
        options=[{'label': t, 'value': t} for t in df['type'].dropna().unique()],
        value=None,
        placeholder="Filter berdasarkan tipe anime...",
        style={'marginBottom': '20px'}
    ),

    dcc.Slider(
        id='year-slider',
        min=df['year'].min(),
        max=df['year'].max(),
        step=1,
        value=df['year'].max(),
        marks={str(year): str(year) for year in sorted(df['year'].unique())},
        tooltip={"placement": "bottom", "always_visible": True}
    ),

    dcc.Graph(id='score-vs-members'),

    dcc.Graph(
        figure=px.pie(df, names='type', title='Distribusi Tipe Anime')
    ),

    dcc.Graph(
        figure=px.histogram(df, x='year', nbins=20, title='Distribusi Tahun Rilis Anime')
    ),

    dcc.Graph(
        figure=px.bar(
            df.sort_values(by='score', ascending=False).head(10),
            x='title', y='score', color='type',
            title='Top 10 Anime Berdasarkan Skor',
            text_auto='.2s'
        ).update_layout(xaxis_tickangle=-45)
    ),

    dcc.Graph(
        figure=px.scatter(df, x='episodes', y='score', size='members', color='type',
                          hover_name='title', title='Jumlah Episode vs Skor Rating', log_x=True)
    )
])

# Callback interaktif
@app.callback(
    Output('score-vs-members', 'figure'),
    [Input('type-dropdown', 'value'),
     Input('year-slider', 'value')]
)
def update_scatter(selected_type, selected_year):
    filtered = df[df['year'] == selected_year]
    if selected_type:
        filtered = filtered[filtered['type'] == selected_type]
    fig = px.scatter(filtered, x='members', y='score', color='type',
                     size='episodes', hover_name='title',
                     title=f'Score vs Jumlah Member ({selected_year})')
    fig.update_layout(xaxis_title='Jumlah Member', yaxis_title='Score')
    return fig

# Run app
if __name__ == '__main__':
    app.run(debug=True, port = 8055)
